In [17]:
import os
from dotenv import load_dotenv
import yfinance as yf
from typing import List, Dict, Any
import requests
from bs4 import BeautifulSoup
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, DuckDuckGoSearchRun, StructuredTool, BaseTool
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage
from datetime import datetime, timedelta
from pydantic import BaseModel, Field

# load_dotenv()

# # Make sure to set your OpenAI API key in your environment variables
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

class StockInput(BaseModel):
    ticker: str = Field(..., description="The stock ticker symbol")

def get_stock_price(ticker: str) -> str:
    try:
        stock = yf.Ticker(ticker)
        end_date = datetime.now()
        start_date = end_date - timedelta(days=30)
        return stock.history(start=start_date, end=end_date).to_csv()
    except Exception as e:
        return f"Error retrieving stock price for {ticker}: {str(e)}"

def get_stock_news(ticker: str) -> List[str]:
    try:
        stock = yf.Ticker(ticker)
        news = stock.news
        one_month_ago = datetime.now() - timedelta(days=30)
        recent_news = [item for item in news if datetime.fromtimestamp(item['providerPublishTime']) > one_month_ago]
        return list(map(lambda x: x["link"], recent_news))
    except Exception as e:
        return [f"Error retrieving news for {ticker}: {str(e)}"]

def get_income_statement(ticker: str) -> str:
    try:
        stock = yf.Ticker(ticker)
        return stock.quarterly_income_stmt.iloc[:, 0].to_csv()
    except Exception as e:
        return f"Error retrieving income statement for {ticker}: {str(e)}"

def get_balance_sheet(ticker: str) -> str:
    try:
        stock = yf.Ticker(ticker)
        return stock.quarterly_balance_sheet.iloc[:, 0].to_csv()
    except Exception as e:
        return f"Error retrieving balance sheet for {ticker}: {str(e)}"

def get_insider_transactions(ticker: str) -> str:
    try:
        stock = yf.Ticker(ticker)
        transactions = stock.insider_transactions
        one_month_ago = datetime.now() - timedelta(days=30)
        recent_transactions = transactions[transactions.index > one_month_ago]
        return recent_transactions.to_csv()
    except Exception as e:
        return f"Error retrieving insider transactions for {ticker}: {str(e)}"

def scrape_website(url: str) -> str:
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text()
    except Exception as e:
        return f"Error scraping website {url}: {str(e)}"

tools = [
    StructuredTool.from_function(
        func=get_stock_price,
        name="StockPrice",
        description="Get stock price history for the last month for a given ticker.",
        args_schema=StockInput
    ),
    StructuredTool.from_function(
        func=get_stock_news,
        name="StockNews",
        description="Get stock news URLs for the last month for a given ticker.",
        args_schema=StockInput
    ),
    StructuredTool.from_function(
        func=get_income_statement,
        name="IncomeStatement",
        description="Get company's most recent quarterly income statement for a given ticker.",
        args_schema=StockInput
    ),
    StructuredTool.from_function(
        func=get_balance_sheet,
        name="BalanceSheet",
        description="Get company's most recent quarterly balance sheet for a given ticker.",
        args_schema=StockInput
    ),
    StructuredTool.from_function(
        func=get_insider_transactions,
        name="InsiderTransactions",
        description="Get insider transactions for the last month for a given ticker.",
        args_schema=StockInput
    ),
    Tool(
        name="WebScraper",
        func=scrape_website,
        description="Scrape content from a given website URL. Input should be a single URL."
    ),
    DuckDuckGoSearchRun()
]

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

system_message = SystemMessage(content="""You are a stock market analyst. Your goal is to provide comprehensive analysis of stocks based on the last month of data.

When using stock-related tools (StockPrice, StockNews, IncomeStatement, BalanceSheet, InsiderTransactions), always provide the input as a JSON object with a 'ticker' key. For example:

- Correct: StockPrice[{"ticker": "AAPL"}]
- Incorrect: StockPrice[AAPL]

For the WebScraper tool, provide the input as a single URL string.

For the Search tool, provide the query as a single string:
- Correct: Search[latest news about Apple stock]
- Incorrect: Search[['Apple stock'], {tags: ['news']}]

Analyze the stock thoroughly, considering technical indicators, financial statements, news, and market sentiment. Focus on data from the last month for your analysis.""")

agent = initialize_agent(
    tools,
    ChatOpenAI(temperature=0, model="gpt-4o-mini"),
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    memory=memory,
    agent_kwargs={
        "system_message": system_message,
        "extra_prompt_messages": [MessagesPlaceholder(variable_name="chat_history")]
    }
)

def analyze_stock(company: str) -> str:
    return agent.run(f"Provide a comprehensive analysis of {company} stock, including technical analysis, financial health, recent news, and an investment recommendation. Base your analysis on data from the last month.")

if __name__ == "__main__":
    analysis = analyze_stock("Salesforce")
    print("\nStock Analysis:")
    print(analysis)

/var/folders/gh/6gsh3jws6nx9x0d7wq7516p40000gn/T/ipykernel_76158/1536103272.py:112: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent = initialize_agent(




> Entering new AgentExecutor chain...

Invoking: `StockPrice` with `{'config': {'tags': ['CRM']}}`


Error retrieving stock price for {'tags': ['CRM']}: 'dict' object has no attribute 'upper'
Invoking: `StockPrice` with `{'config': {'tags': ['CRM']}}`
responded: I encountered an error while retrieving the stock price for Salesforce (CRM). Let me try again.

Error retrieving stock price for {'tags': ['CRM']}: 'dict' object has no attribute 'upper'
Invoking: `StockPrice` with `{'config': {'tags': ['SALESFORCE']}}`


Error retrieving stock price for {'tags': ['SALESFORCE']}: 'dict' object has no attribute 'upper'
Invoking: `IncomeStatement` with `{'args': ['CRM'], 'config': {}}`
responded: It seems there is an issue with retrieving the stock price for Salesforce (CRM). Let me proceed with analyzing other aspects and provide a comprehensive analysis based on available information.



ToolException: Too many arguments to single-input tool IncomeStatement.
                Consider using StructuredTool instead. Args: [['CRM'], {}]